# Import Libries

In [22]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics as mt

# Load Dataset

In [2]:
# 1. Prepare os dados
df = pd.read_csv('train.csv')
df.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar


In [23]:
df.loc[:,'limite_adicional'].unique()

array(['Negar', 'Conceder'], dtype=object)

In [3]:
# Conjunto de dados X (features) e y (rótulos/classes)
feactures = ['idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros', 'investe_exterior', 'pessoa_polit_exp']
label = 'limite_adicional'       

In [33]:
# contagem de quantos negar/conceder
contagem = df.loc[:,'limite_adicional'].value_counts()
# Porcentagem da quantidade Negar/Conceder
porcentagem = df.loc[:,'limite_adicional'].value_counts(normalize=True)
print(contagem)
print(porcentagem)

Negar       7995
Conceder    1505
Name: limite_adicional, dtype: int64
Negar       0.841579
Conceder    0.158421
Name: limite_adicional, dtype: float64


In [4]:
# 2. Divida os dados em conjunto de treinamento
x_train = df.loc[:,feactures]
y_train = df.loc[:,label]

In [5]:
# Supondo que 'x_train' seja o DataFrame que contém seus dados de treinamento e devemos excluir dados nao relevantes ou que nao seja float/int
drop_n_relevante = ['investe_exterior', 'pessoa_polit_exp']
x_train = x_train.drop(drop_n_relevante, axis=1)

# Treinamento

In [37]:
# 2. Defina uma lista de valores K para experimentar
k_values = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

# 3. Crie uma instância do classificador KNN
knn = KNeighborsClassifier()

# 4. Defina os parâmetros para a busca por hiperparâmetros
# param_grid = {'n_neighbors': k_values}

# 5. Realize a busca por hiperparâmetros usando validação cruzada
grid_search = GridSearchCV(knn, param_grid, cv=7)  # cv é o número de folds na validação cruzada
grid_search.fit(x_train, y_train)

# 6. Obtenha os resultados da busca por hiperparâmetros
print("Melhores parâmetros: ", grid_search.best_params_)
print("Melhor acurácia média: ", grid_search.best_score_)


Melhores parâmetros:  {'n_neighbors': 21}
Melhor acurácia média:  0.83789457102763


### Predicao

In [39]:
# 5. Faça previsões no conjunto de teste
y_pred =grid_search.predict(x_train)

In [43]:
# copy dataframe
df_result = df.copy()# veio todas
# criar uma col  classificacao
df_result['classificacao'] = y_pred

# 4.0 Performace

# Matriz da confusao

In [55]:
df_result['acertos'] = df_result.loc[:,['id_cliente','limite_adicional','classificacao']].apply(lambda x: 1 if x['limite_adicional'] == x['classificacao'] else 0, axis=1)

In [56]:
df_result.loc[:,['id_cliente','idade','limite_adicional','classificacao','acertos'] ].sample(20)

,id_cliente,idade,limite_adicional,classificacao,acertos
1543,12274,39,Negar,Negar,1
1424,2389,28,Negar,Negar,1
3060,12430,30,Negar,Negar,1
7060,306,506,Negar,Negar,1
5196,4223,30,Conceder,Conceder,1
292,10387,31,Negar,Negar,1
6703,4477,39,Conceder,Negar,0
7285,1815,28,Negar,Negar,1
8577,9120,35,Negar,Negar,1
836,10366,33,Conceder,Negar,0


### 4.1 Acuracia

In [ ]:
# contagem de quantos negar/conceder
contagem = df.loc[:,'limite_adicional'].value_counts()
# Porcentagem da quantidade Negar/Conceder
porcentagem = df.loc[:,'limite_adicional'].value_counts(normalize=True)
print(contagem)
print(porcentagem)

Negar       7995
Conceder    1505
Name: limite_adicional, dtype: int64
Negar       0.841579
Conceder    0.158421
Name: limite_adicional, dtype: float64


In [14]:
# 6. Avalie o desempenho do modelo
accuracy = mt.accuracy_score(y_train, y_pred)
print("Acurácia: %.2f%%" % (accuracy * 100))

Acurácia: 85.91%


### 4.2 Precisao

In [15]:
mt.precision_score(x_train, y_train)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

### 4.3 Recall

Exercícios
1. Rafaça o código de treinamento da aula 12: “K-Nearest Neighbors - Prática” no seu computador usando o Jupyter Notebook ou o Google Colabs.
2. Retreino o algoritmo com os seguintes valores para K: [3, 5, 7, 9, 11, 13, 15, 17, 19 e 21] e anote a acurácia.
3. Qual o problema principal de usar a métrica acurácia? Escreve um exemplo hipotético, no qual o problema acontece.
4. Explique com um pequeno texto ilustrando a diferença entre a métrica de Precision e Recall e mostrando quando usa deve ser escolhida em relação a outra.
5. Escreve um trecho de código que automatize o treinamento do algoritmo K-NN, a fim de encontrar o melhor valor para K, do exercício 2.
6. Escreva um pequeno texto, explicando as 6 denominações da matriz de confusão: P, N, TP, FN, FP e TN
7. No conjunto de dados usado na aula 12: “K-Nearest Neighbors - Prática”, existe alguma variável que fere as premissas do K-NN? Se sim ou não, explique.
8. Faça a seguinte bateria de testes
    8.1 Class balanceada originalmente:
1. Faça a matriz de confusão, calcule a acurácia, recall e precision do conjunto de dados original
2. Anote os resultados. 
    8.2 Classe balanceada:
1. Mantenha a proporção de 50% das linhas da planilha de dados com exemplos da classe “Conceder” e 50% com a classe “Negar”.
2. Faça a matriz de confusão, calcule a acurácia, recall e precision.
3. Anote os resultados.
    8.3 Classe desbalanceada 90/10
1. Mantenha a proporção de 90% das linhas da planilha de dados com exemplos da classe “Conceder” e 10% com a classe “Negar”.
2. Faça a matriz de confusão, calcule a acurácia, recall e precision.
3. Anote os resultados.
    8.4 Refaça o exercícios 8.3 com as seguintes variações: 80/20, 70/30 e 60/40
    8.5 Crie um tabela de comparação entre os resultados das 3 métricas ( acurácia, recall e precision ) para cada uma das proporções ( 50/50, 90/10, 80/20, 70/30, 60/40 ) e responda as seguintes perguntas:
1. Como a métrica da acurácia se comporta com a variação do desbalanceamento do conjunto de dados?
2. O que acontece com a métrica “Precison” e “Recall” a medida que os conjunto de dados tendem ao balanceamento de 50/50?
3. Observando as respostas anteriores, quais são as ações que aumentam ou diminuem a métrica de “Precision” ou “Recall” de um problema de negócio?
    8.6 Escreva um artigo para blog da Comunidade DS, descrevendo o seu aprendizado em relação ao comportamento das métricas de Precision, Recall e Acurácia a partir dos seus experimentos com os dados desbalanceados.